# Gramática y parser de constituyentes (CFG) para el bribri
**Grammar and CFG parser for Bribri**

Versión b01, 20211203<br>Rolando Coto-Solano

El programa recibe dos inputs: (i) una oración en bribri y (ii) las partes de la palabra de cada palabra. El output es un parseo de constituyentes.

The program receives two inputs: (i) a sentence in Bribri and (iii) the POS for each word. The output is a constituent parse.

## Gramática CFG del bribri
**Bribri CFG Grammar**

In [1]:
import nltk

In [2]:
grammarBribri = nltk.CFG.fromstring("""
  
    S      -> NP VP | VP | VP NP | ADVP NP VP | PP NP VP | PP PP NP VP | ADVP PP NP VP  
    S      -> ADVCLP PP NP VP | ADVCLP PP PP NP VP | ADVCLP PP VP NP
    
    NP     -> N | PRON | N ERG | PRON ERG | PRON N | PRON N ERG | N N | PRON N N | PRON N N N
    NP     -> N ADJP
    NP     -> N DEM | N N DEM | N NUM | N ADJP DEM | N ADJP DEM ERG
    
    CONJP  -> NP1 CONJ NP2
    NP1    -> NP
    NP2    -> NP
    
    VP     -> COP ADJP | COP NP | COP PP
    VP     -> COP CONJP
    VP     -> ADV COP ADJP | ADV COP NP | ADV COP PP
    VP     -> VRIGHT NP | VRIGHT | VRIGHT CP | VRIGHT PP | VRIGHT PP CONJPP | VRIGHT NUM PP | VRIGHT PP NP | VRIGHT ADVP
    VP     -> NP VRIGHT NP | NP VRIGHT ADVP NP
    VP     -> VRIGHT CP ADVCLP | VRIGHT ADVCLP
    VP     -> NP V | NP V ADVP | NP V NUM | NP V PP | PP NP V
    VP     -> V | V ADVP | ADVP V | ADVP V ADVP | ADVP NP V | V PP | PP V | V NUM | V NUM PP | V ADVP PP
    VP     -> PP NP V PP ADVP | PP NP V PP PP | NP V PP PP
    VP     -> POSIT PP | POSIT ADVP | POSIT NUM | POSIT NUM PP
    VP     -> V CP | ADV V CP
    VP     -> NP VINF | NP VINF ADVP | PP NP VINF
    VP     -> VINF NUM | VINF | VINF PP | PP VINF | VINF PP PP
    VP     -> V SUBORDP
    VP     -> V ADVP ADVCLP
    VP     -> ADVP V POSIT PP | V POSIT

    CP      -> VP
    SUBORDP -> SUBORDMARK NP VP

    ADVCLP -> ADVCL NP VP
    ADVCLP -> NP VP | VP

    PP     -> NP POSTP  | NP ADVP POSTP | CONJP POSTP
    CONJPP -> PP1 CONJ PP2
    PP1    -> PP
    PP2    -> PP

    ADJP   -> ADJ | ADJ ADJ | ADJ ADVP
    ADVP   -> ADV | ADV ADV | NP PART | ADV PART | PART
    ADVCLP -> ADVCL

    N      -> "leaf:noun" | "leaf:nprop"
    DEM    -> "leaf:dem-pronoun"
    ERG    -> "leaf:ergative"
    CONJ   -> "leaf:conjunction"
    V      -> "leaf:verb"
    VRIGHT -> "leaf:verb-objright"
    VINF   -> "leaf:verb-infinitive"
    POSIT  -> "leaf:positional"
    PRON   -> "leaf:pronoun" | "leaf:dem-pronoun"
    COP    -> "leaf:copula"
    ADJ    -> "leaf:adjective" | "leaf:quantifier"
    ADV    -> "leaf:adverb" | "leaf:adjective" | "leaf:wh-adv" | "leaf:negative" | "leaf:ideophone" | "leaf:quantifier"
    POSTP  -> "leaf:postposition"
    NUM    -> "leaf:numeral"
    ADVCL  -> "leaf:subord-clause"
    SUBORDMARK -> "leaf:subord-mark"
    PART   -> "leaf:particle"

  """)

## Funciones para parsing
**Parsing functions**

### Recorrido a través de los árboles
**Tree traverse**

In [18]:
#https://stackoverflow.com/questions/28681741/find-a-path-in-an-nltk-tree-tree

def get_lca_length(location1, location2):
    i = 0
    while i < len(location1) and i < len(location2) and location1[i] == location2[i]:
        i+=1
    return i

def get_labels_from_lca(ptree, lca_len, location):
    labels = []
    for i in range(lca_len, len(location)):
        labels.append(ptree[location[:i]].label())
    return labels

def findPath(ptree, inIndex1, inIndex2):
    leaf_values = ptree.leaves()
    leaf_index1 = inIndex1
    leaf_index2 = inIndex2

    location1 = ptree.leaf_treeposition(leaf_index1)
    location2 = ptree.leaf_treeposition(leaf_index2)

    #find length of least common ancestor (lca)
    lca_len = get_lca_length(location1, location2)

    #find path from the node1 to lca

    labels1 = get_labels_from_lca(ptree, lca_len, location1)
    #ignore the first element, because it will be counted in the second part of the path
    result = labels1[1:]
    #inverse, because we want to go from the node to least common ancestor
    result = result[::-1]

    #add path from lca to node2
    result = result + get_labels_from_lca(ptree, lca_len, location2)
    return result

### Llamar a NTLK para construir el árbol
**Make NLTK call to get tree**

In [4]:
import time
import nltk
nltk.download('treebank')

# This function gets the possible parses for a tree.
def getTree(phraseToBeParsed, grammar, words, pos, foma, squareBrackets=False, verbose = 0):

    t = ""
    rejectTree = 0
    counter = 1

    sent = phraseToBeParsed.split()
    rd_parser = nltk.RecursiveDescentParser(grammar, trace=0)
        
    for tree in rd_parser.parse(sent):

      try:
        if (verbose == 1): print("*** i'm trying to parse tree " + str(counter) + " ***")
        rejectTree = rejectCFGDueToBribriRules(tree, words, pos, foma, verbose = verbose)
        if (rejectTree == 0): t = str(tree) + "\n"
        if (verbose == 1): print(str(tree).replace("(", "[").replace(")","]"))
        counter += 1
      except RuntimeError:
        pass

    return t

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


### Comenzar el parseo con CFG
**Start CFG Parse**


In [5]:
def getCFGParse(inputWords, inputPOS, inputFoma, inputCFG = "", convertSofiaOrthography = 0, verbose = 0):

  resTree = ""
  posLeafTree = ""
  posNumberedTree = ""

  constituentSeparator = ")"
  
  for i in range(0, len(inputPOS)):
    if (len(inputPOS[i]) < 5) or (inputPOS[i][0:5] != "leaf:"):
      inputPOS[i] = "leaf:" + inputPOS[i]

  posAsLine = " ".join(inputPOS)
  
  try:
    if (inputCFG == ""):
      resTree = getTree(posAsLine, grammarBribri, inputWords, inputPOS, inputFoma, verbose = verbose)
    else:
      if (inputCFG[0:1] == "["): inputCFG = inputCFG.replace("[", "(").replace("]", ")")
      tempTree = nltk.tree.Tree.fromstring(inputCFG)
      for leafPos in tempTree.treepositions('leaves'):
        tempTree[leafPos] = "leaf:" + tempTree[leafPos]
      resTree = str(tempTree)
  except Exception as e:
    print("Exception: CFG parsing failed")
    print(e)

  if (resTree != ""):

    posLeafTree = nltk.tree.Tree.fromstring(resTree)
    posNumberedTree = resTree

    if (resTree != ""):
      # for every word in inputWords: (i) find the leftmost leaf (ii) find
      # it's beginning and end indeces, and (iii) replace that with the word
      for i in range(0, len(inputWords)):
        indexStartWord = resTree.find("leaf:")
        indexBracket = resTree.find(constituentSeparator, indexStartWord)
        indexEndWord = indexBracket - 1
        wordInLeaf = inputWords[i]
        if (convertSofiaOrthography == 1): wordInLeaf = convertToHumanSpelling(wordInLeaf)
        resTree = resTree[:indexStartWord] + wordInLeaf + resTree[indexEndWord + 1:]
      for i in range(0, len(inputWords)):
        indexStartWord = posNumberedTree.find("leaf:")
        indexBracket = posNumberedTree.find(constituentSeparator, indexStartWord)
        indexEndWord = indexBracket - 1
        wordInLeaf = inputWords[i]
        if (convertSofiaOrthography == 1): wordInLeaf = convertToHumanSpelling(wordInLeaf)
        posNumberedTree = posNumberedTree[:indexStartWord] + str(i+1) + "/" + wordInLeaf + posNumberedTree[indexEndWord + 1:]

      resTree = nltk.tree.Tree.fromstring(resTree)
      posNumberedTree = nltk.tree.Tree.fromstring(posNumberedTree)

    for i in range(0, len(inputPOS)):
      inputPOS[i] = inputPOS[i].replace("leaf:", "")

  else:
    print("ERROR: I couldn't produce a CFG parse")

  return (resTree)

### Rechazar árboles CFG por reglas internas de la lengua
Reject CFG trees because of language-internal rules

In [20]:
def rejectCFGDueToBribriRules(inTree, inWords, inPOS, inFoma, verbose = 0):

  reject = 0

  # =====================================================================
  # Reject sentences without a subject. A sentence where:
  # (i) the main verb has an absolutive, but
  # (ii) the absolutive is not in an NP that is a sister to VP
  # E.g.: (S (VP (NP  (the only NP got swallowed into the VP)
  # =====================================================================

  isThereErgativeSubject = 0
  areThereNounsBeforeV = 0
  isThereASubject = 0

  for i in range(0,len(inPOS)):
    
    if (inPOS[i] == "leaf:verb"):
      indexVerb = i
      for j in range(0,len(inPOS)):
        
        # First, is there an ergative subject?
        if (inPOS[j] == "leaf:ergative"):
          path = findPath(inTree, indexVerb, j)
          if ("S" in path and "CP" not in path and "PP" not in path):
            isThereErgativeSubject = 1

    # Second, are there any subjects?
    if (inPOS[i] == "leaf:verb" or inPOS[i] == "leaf:verb-objright" or inPOS[i] == "leaf:positional"):
      indexVerb = i
      for j in range(0,len(inPOS)):
        if (inPOS[j] == "leaf:noun" or inPOS[j] == "leaf:nprop" or inPOS[j] == "leaf:pronoun" or inPOS[j] == "leaf:dem-pronoun"):
          path = findPath(inTree, indexVerb, j)
          if ("S" in path): isThereASubject = 1


  # Third, do I have any nouns to the left of the verb?
  indexFirstV = -1
  indexFirstV = str(inTree).find("(V ")
  if (indexFirstV == -1):
    indexFirstV = str(inTree).find("(VRIGHT ")
    if (indexFirstV == -1):
      indexFirstV = str(inTree).find("(VERB-OBJRIGHT ")
      if (indexFirstV == -1):
        indexFirstV = str(inTree).find("(POSIT ")
  indexFirstNP = -1
  indexFirstNP = str(inTree).find("(NP")
  if (indexFirstNP != -1 and indexFirstNP < indexFirstV):
    areThereNounsBeforeV = 1

  if (areThereNounsBeforeV != 0 and isThereErgativeSubject == 0 and isThereASubject == 0):
    reject = 1
    if (verbose == 1): print("*** Tree rejected because of lack of subject")

  # =====================================================================
  # Reject sentences without a verb. A sentence where:
  # (i) The sentence does have a VP connected to S, but (ii) the VP 
  # does not contain a finite verb
  # =====================================================================

  if (reject == 0):

    doesTreeHaveVP = 0
    doesVPHaveFiniteVerb = 0

    for child1 in inTree:
      child1Label = child1.label()
      if (child1Label == "VP"):
        doesTreeHaveVP = 1
        for child2 in child1:
          child2Label = child2.label()
          if (child2Label == "V" or child2Label == "VRIGHT" or child2Label == "POSIT") or child2Label == "COP":
            doesVPHaveFiniteVerb = 1
    
    if (doesTreeHaveVP == 1 and doesVPHaveFiniteVerb == 0):
      reject = 1
      if (verbose == 1): print("*** Reject because the sentence as a whole has a VP but it doesn't have a finite verb as its root")

  # =====================================================================
  # Reject a sentence where the adverbial clause stepped into 
  # the main clause (where the e'ta is associated with the
  # adverbial clause and not the main clause)
  # =====================================================================

  if (reject == 0):

    doesPhraseHaveEta = 0
    indexesEta = []

    # See if the phrase "e' tã" ("then, therefore") exists in the sentence
    for i in range (0, len(inWords)):
      if (inWords[i] == "e'"):
        if (i+1 < len(inWords)):
          if (inWords[i+1] == "tã"):
            doesPhraseHaveEta = 1
            indexesEta.append(i)

    if (doesPhraseHaveEta == 1):

      for eta in indexesEta:

        reversePOS = inPOS[::-1]
        reverseIndexEta = len(reversePOS) - eta - 1

        # See if there is a preceding verb
        foundNearestPreviousVerb = 0
        indexNearestPreviousVerb = -1
        for i in range(reverseIndexEta + 1, len(reversePOS)):
          if (foundNearestPreviousVerb == 0):
            if (reversePOS[i] == "leaf:verb" or reversePOS[i] == "leaf:verb-objright" or reversePOS[i] == "leaf:positional" or reversePOS[i] == "leaf:verb-infinitive"):
              foundNearestPreviousVerb = 1
              indexNearestPreviousVerb = len(reversePOS) - i - 1
        
        if (foundNearestPreviousVerb == 1):
          pathPreviousVP = findPath(inTree, eta, indexNearestPreviousVerb)

        # See if there is a preceding noun
        foundNearestPreviousNoun = 0
        indexNearestPreviousNoun = -1
        for i in range(reverseIndexEta + 1, len(reversePOS)):
          if (foundNearestPreviousNoun == 0):
            if (reversePOS[i] == "leaf:noun" or reversePOS[i] == "leaf:nprop"):
              foundNearestPreviousNoun = 1
              indexNearestPreviousNoun = len(reversePOS) - i - 1

        if (foundNearestPreviousNoun == 1):
          pathPreviousNP = findPath(inTree, eta, indexNearestPreviousNoun)

        # See if there is a following verb
        foundNearestFollowingVerb = 0
        indexNearestFollowingVerb = -1
        for i in range(eta + 2, len(inPOS)):
          if (foundNearestFollowingVerb == 0):
            if (inPOS[i] == "leaf:verb" or inPOS[i] == "leaf:verb-objright" or inPOS[i] == "leaf:positional" or inPOS[i] == "leaf:verb-infinitive"):
              foundNearestFollowingVerb = 1
              indexNearestFollowingVerb = i

        if (foundNearestFollowingVerb == 1):
          pathFollowingVP = findPath(inTree, eta, indexNearestFollowingVerb)

        if (foundNearestPreviousVerb == 1 and foundNearestFollowingVerb == 1 and len(pathPreviousVP) > 0 and len(pathFollowingVP) > 0):
          
          try: indexSToLeft = pathPreviousVP.index("S")
          except: indexSToLeft = -1
          try: indexAdvCLPtoLeft = pathPreviousVP.index("ADVCLP")
          except: indexAdvCLPtoLeft = -1
          try: indexSToRight = pathFollowingVP.index("S")
          except: indexSToRight = -1
          try: indexAdvCLPtoRight = pathFollowingVP.index("ADVCLP")
          except: indexAdvCLPtoRight = -1

          if (indexAdvCLPtoLeft != -1 and indexSToLeft > indexAdvCLPtoLeft):
            reject = 1
            if (verbose == 1): print("*** Tree rejected because of bad edge of adverbial clause (e' ta in a subordinate to the left of the main clause)")
          elif (indexAdvCLPtoRight != -1 and indexSToRight > indexAdvCLPtoRight):
            reject = 1
            if (verbose == 1): print("*** Tree rejected because of bad edge of adverbial clause (e' ta in a subordinate to the right of the main clause)")

        if (foundNearestPreviousNoun == 1 and foundNearestPreviousVerb == 1 and len(pathPreviousNP) > 0):
          if ("ADVCLP" not in pathPreviousNP):
            reject = 1
            if (verbose == 1): print("*** Tree rejected because of bad edge of adverbial clause (noun from subord clause linked to e' ta)")

  # =====================================================================
  # Reject a sentence where the ADVCLP (adverbial phrase) does not
  # contain a finite verb (these should be in CPs)
  # =====================================================================

  if (reject == 0):

    if "ADVCLP" in str(inTree):

      finiteVerbs = []
      nonfiniteVerbs = []
      for i in range(0,len(inPOS)):
        if (inPOS[i] == "leaf:verb" or inPOS[i] == "leaf:positional" or inPOS[i] == "leaf:verb-objright"): finiteVerbs.append(1)
        else: finiteVerbs.append(0)
        if (inPOS[i] == "leaf:verb-infinitive"): nonfiniteVerbs.append(1)
        else: nonfiniteVerbs.append(0)

      for i in range(0, len(finiteVerbs)):
        for j in range(0, len(nonfiniteVerbs)):
          if (finiteVerbs[i] == 1 and nonfiniteVerbs[j] == 1):
            path = findPath(inTree, i, j)
            if 'CP' not in path:
              reject = 1
              if (verbose == 1): print("*** Tree rejected an infinitive was not contained in a CP")
            

  return(reject)

### parseBribri (función inicial)
Recibe una oración y la POS para cada palabra, y pone cada una en un vector. Luego le pasa esta información de getCFGParse.

In [21]:
def parseBribri(sentence, pos):
  resWords = sentence.split(" ")
  resPOS = pos.split(" ")
  cfgParse = getCFGParse(resWords, resPOS, "", "", verbose = 0)
  cfgParse = str(cfgParse)
  cfgParse = cfgParse.replace("(","[")
  cfgParse = cfgParse.replace(")","]")
  return cfgParse

## Ejemplos de parsing
Parsing examples

Puede usar un sitio como http://mshang.ca/syntree/ para visualizar los árboles.<br>
You can use a site like http://mshang.ca/syntree/ to visualize the trees.

In [19]:
resParse = parseBribri("Ye' tö ù sú̠", "pronoun ergative noun verb")    # Yo vi la casa, I saw the house
print(resParse)

[S [NP [PRON Ye'] [ERG tö]] [VP [NP [N ù]] [V sú̠]]]


In [22]:
resParse = parseBribri("Wë́pa ië'te̠n ù a̠", "noun positional noun postposition")    # Los hombres están (de pie) en la casa, The men are (standing) in the house
print(resParse)

[S [NP [N Wë́pa]] [VP [POSIT ië'te̠n] [PP [NP [N ù]] [POSTP a̠]]]]


In [17]:
resParse = parseBribri("Chìchi dör sarûrû", "noun copula adjective")    # El perro es blanco, The dog is white
print(resParse)

[S [NP [N Chìchi]] [VP [COP dör] [ADJP [ADJ sarûrû]]]]
